In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import pickle
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add

Using TensorFlow backend.


In [2]:
model=load_model("model_19.h5")

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [119]:
temp_model=ResNet50(weights="imagenet",input_shape=(224,224,3))
temp_model.summary()


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [120]:
resnet_model=Model(temp_model.input,temp_model.layers[-2].output)

In [121]:
def functiontoPreprocessImage(img):
    
    img=image.load_img(img,target_size=(224,224))
    img=image.img_to_array(img)
    
    #ResNet accepts a 4D Tensor so we do not feed an single Image but Batch of Image thus we expand :
    #(1,224,224,3)
    img=np.expand_dims(img,axis=0)
    
    #Normalizing the Data as per ResNet50: preprocess_input is Resnet's Inbuilt Function
    img=preprocess_input(img)
    
    return img

In [122]:
def encode_image(img):
    
    img= functiontoPreprocessImage(img)
    feature_vector=resnet_model.predict(img)
    
    feature_vector=feature_vector.reshape(1,feature_vector.shape[1])
#     print(feature_vector.shape)
    
    return feature_vector

In [135]:
encode=encode_image('football.jpg')
encode.shape

(1, 2048)

In [136]:
with open("word_to_idx.pkl","rb") as w2i:
    word_to_idx=pickle.load(w2i)

In [137]:
with open("idx_to_word.pkl","rb") as i2w:
    idx_to_word=pickle.load(i2w)

In [138]:
print(word_to_idx)

{'dish': 1, 'skateboarders': 2, 'descends': 3, 'dive': 4, 'multicolor': 5, 'come': 6, 'incline': 7, 'sunlight': 8, 'elaborate': 9, 'fireplace': 10, 'sprays': 11, 'folding': 12, 'seaweed': 13, 'stretch': 14, 'sporting': 15, 'flowery': 16, 'stretched': 17, 'cushion': 18, 'sat': 19, 'leashed': 20, 'blocking': 21, 'wakeboarder': 22, 'including': 23, 'leotard': 24, 'headdress': 25, 'boards': 26, 'heavily': 27, 'posts': 28, 'own': 29, 'peeks': 30, 'leaf': 31, 'container': 32, 'overhead': 33, 'parasailing': 34, 'handle': 35, 'facial': 36, 'bouncy': 37, 'limb': 38, 'bottles': 39, 'bread': 40, 'halloween': 41, 'tricycles': 42, 'choppy': 43, 'aqua': 44, 'casts': 45, 'floppy': 46, 'works': 47, 'tights': 48, 'glider': 49, 'calm': 50, 'bounding': 51, 'mess': 52, 'hurdles': 53, 'native': 54, 'wade': 55, 'observes': 56, 'fellow': 57, 'kicked': 58, 'similar': 59, 'snake': 60, 'beverage': 61, 'bearing': 62, 'footballer': 63, 'cheer': 64, 'injured': 65, 'skies': 66, 'pitching': 67, 'studio': 68, 'bunny'

In [139]:
#Prediction

def prediction_caption(Picture):
    
    input_text="startseq"
    max_sentence_len=35
    for i in range(max_sentence_len):
        
        sequence=[word_to_idx[word] for word in input_text.split() if word in word_to_idx ]
        sequence=pad_sequences([sequence],maxlen=max_sentence_len,padding='post')
        
        prediction=model.predict([Picture,sequence])
        prediction=prediction.argmax()
        predicted_word=idx_to_word[prediction]
        input_text+=(' '+predicted_word)
        
        if predicted_word=='endseq':
            break
            
    final_caption = input_text.split()[1:-1]
    final_caption=' '.join(final_caption)
    
    return final_caption

In [140]:
output=prediction_caption(encode)
print(output)

two men in blue uniforms are playing soccer
